In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
# Define the transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to the desired size
    transforms.ToTensor(),           # Convert the image to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization in imagenet

])

# Define the path to your dataset folder
path = "images"

# Load the dataset
dataset = ImageFolder(root=path, transform=transform)


In [3]:
batch_size = 256  # Adjust as needed

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [4]:
# Load a pre-trained AlexNet model
alexnet = models.alexnet(weights="AlexNet_Weights.DEFAULT")

# Modify the output layer to match the number of classes (5)
num_classes = 5  # Adjust to the number of classes in your dataset
alexnet.classifier[6] = nn.Linear(4096, num_classes)


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(alexnet.parameters(), lr=0.001)


In [6]:
total_correct = 0
total_samples = 0

num_epochs = 10


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet.to(device)

# Training 
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0

    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    # Wrap train_loader with tqdm for a progress bar
    
    train_loader_with_progress = tqdm(train_loader)

    for inputs, labels in train_loader_with_progress:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Compute accuracy for this batch
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        # Update the tqdm progress bar with both loss and accuracy
        train_loader_with_progress.set_postfix({"Loss": loss.item(), "Accuracy": total_correct / total_samples * 100.0})

    average_loss = running_loss / len(train_loader)
    # print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {average_loss:.4f}")

# Training accuracy
training_accuracy = total_correct / total_samples * 100.0
print(f"Training Accuracy: {training_accuracy:.2f}%")

Epoch [1/5]


100%|██████████| 79/79 [03:05<00:00,  2.35s/it, Loss=0.632, Accuracy=62.2]


Epoch [2/5]


100%|██████████| 79/79 [02:56<00:00,  2.23s/it, Loss=0.224, Accuracy=67.3]


Epoch [3/5]


...

Epoch [4/5]


100%|██████████| 79/79 [02:49<00:00,  2.15s/it, Loss=0.175, Accuracy=78.7] 


Epoch [5/5]


100%|██████████| 79/79 [02:46<00:00,  2.10s/it, Loss=0.00938, Accuracy=82] 

Training Accuracy: 82.03%


In [7]:
# Evaluation on the test set
alexnet.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0

# Wrap test_loader with tqdm for a progress bar
test_loader_with_progress = tqdm(test_loader)

for inputs, labels in test_loader_with_progress:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = alexnet(inputs)
    _, predicted = torch.max(outputs, 1)
    total_samples += labels.size(0)
    total_correct += (predicted == labels).sum().item()

# Compute and print the test accuracy
test_accuracy = total_correct / total_samples * 100.0
print(f"Test Accuracy: {test_accuracy:.2f}%")

100%|██████████| 20/20 [00:43<00:00,  2.16s/it]

Test Accuracy: 95.38%
